# Requirements


1.   torch : Le package principal de PyTorch pour les tenseurs.
2.   torch.nn : Contient les modules de réseaux neuronaux.
3. torch.optim : Fournit des algorithmes d'optimisation.
4. torch.nn.functional : Contient des fonctions comme relu et linear, qui sont des versions fonctionnelles des modules.



In [1]:
!pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F



1. TinyLlama : Un modèle de réseau neuronal simple avec deux couches linéaires (fc1 et fc2).

2. fc1 : Une couche linéaire prenant une entrée de dimension 768 et produisant une sortie de dimension 1024.
3. fc2 : Une autre couche linéaire qui prend une entrée de 1024 dimensions et produit une sortie de 2 dimensions, représentant les classes de sortie pour la classification binaire.



In [5]:
class TinyLlama(nn.Module):
    def __init__(self):
        super(TinyLlama, self).__init__()
        self.fc1 = nn.Linear(768, 1024)
        self.fc2 = nn.Linear(1024, 2)  
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

1. LoRALayer : Ajoute des matrices de faible rang (A et B) à une couche linéaire pour adapter le modèle préentraîné à une nouvelle tâche.
2. original_layer : La couche linéaire d'origine que nous voulons adapter.
3. lora_A et lora_B : Matrices de faible rang qui sont des paramètres entraînables ajoutés pour ajuster la sortie de la couche originale.
4. forward : Calcule d'abord la sortie d'origine avec original_layer, puis ajoute l'ajustement calculé à l'aide des matrices lora_A et lora_B.

In [6]:
class LoRALayer(nn.Module):
    def __init__(self, original_layer, input_dim, r=4):
        super(LoRALayer, self).__init__()
        self.original_layer = original_layer
        self.input_dim = input_dim
        self.r = r
        self.lora_A = nn.Parameter(torch.randn(input_dim, r))
        self.lora_B = nn.Parameter(torch.randn(r, original_layer.weight.size(0)))

    def forward(self, x):
        original_output = self.original_layer(x)
        lora_output = torch.mm(torch.mm(x, self.lora_A), self.lora_B)
        return original_output + lora_output

class TinyLlamaWithLoRA(nn.Module):
    def __init__(self):
        super(TinyLlamaWithLoRA, self).__init__()
        self.fc1 = LoRALayer(nn.Linear(768, 1024), input_dim=768)
        self.fc2 = nn.Linear(1024, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x



1. QLoRALayer : Similaire à LoRALayer, mais inclut une quantification des poids de la couche originale.
2. quantized_weight : Les poids de original_layer sont quantifiés, réduisant la précision pour économiser de la mémoire.
3. Quantification : Convertit les poids en un format de moindre précision (ex: 4-bit).
4. Déquantification et Inférence : Avant de calculer la sortie, les poids sont déquantifiés pour obtenir les valeurs approximatives d'origine.

In [7]:
class QLoRALayer(nn.Module):
    def __init__(self, original_layer, input_dim, r=4):
        super(QLoRALayer, self).__init__()
        self.original_layer = original_layer
        self.input_dim = input_dim
        self.r = r
        self.lora_A = nn.Parameter(torch.randn(input_dim, r))
        self.lora_B = nn.Parameter(torch.randn(r, original_layer.weight.size(0)))
        # Quantification des poids de la couche originale
        self.quantized_weight = torch.quantize_per_tensor(
            original_layer.weight.data, scale=0.1, zero_point=0, dtype=torch.qint8
        )

    def forward(self, x):
        # Déquantification pour l'inférence
        dequantized_weight = self.quantized_weight.dequantize()
        original_output = F.linear(x, dequantized_weight)
        lora_output = torch.mm(torch.mm(x, self.lora_A), self.lora_B)
        return original_output + lora_output


In [9]:
class TinyLlamaWithQLoRA(nn.Module):
    def __init__(self):
        super(TinyLlamaWithQLoRA, self).__init__()
        self.fc1 = QLoRALayer(nn.Linear(768, 1024), input_dim=768)
        self.fc2 = nn.Linear(1024, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def calculate_accuracy(model, inputs, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == labels).sum().item() / labels.size(0)
    return accuracy

In [10]:
train_inputs = torch.randn(100, 768)
train_labels = torch.randint(0, 2, (100,))
test_inputs = torch.randn(20, 768)
test_labels = torch.randint(0, 2, (20,))

In [11]:
model_lora = TinyLlamaWithLoRA()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_lora.parameters(), lr=0.001)
model_lora.train()
for epoch in range(5):  
    optimizer.zero_grad()
    outputs = model_lora(train_inputs)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
accuracy_lora = calculate_accuracy(model_lora, test_inputs, test_labels)

In [12]:
model_qlora = TinyLlamaWithQLoRA()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_qlora.parameters(), lr=0.001)
model_qlora.train()
for epoch in range(5): 
    optimizer.zero_grad()
    outputs = model_qlora(train_inputs)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
accuracy_qlora = calculate_accuracy(model_qlora, test_inputs, test_labels)

print(f"Accuracy LoRA: {accuracy_lora * 100:.2f}%")
print(f"Accuracy QLoRA: {accuracy_qlora * 100:.2f}%")

Accuracy LoRA: 45.00%
Accuracy QLoRA: 50.00%
